In [1]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [4]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 5000000, chunk_overlap= 1000)

# Replace this with our data
#all_splits = text_splitter.split_documents(data)

In [ ]:
vectorstore = Chroma.from_documents(
    documents = all_splits, 
    embedding = OpenAIEmbeddings(),
    persist_directory = "data/chroma_db"
)

retriever = vectorstore.as_retriever()

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0.0)

memory = ConversationBufferMemory(
    llm = llm,
    memory_key = "chat_history",
    return_messages=True
)

chat = ConversationalRetrievalChain.from_llm(
    llm,
    retriever = retriever,
    memory = memory
)

In [ ]:
res = chat("Por qué es tan buen profe Ivan?")

In [ ]:
print(chat["answer"])